# 📊 Exploração Inicial dos Dados - Projeto SentinelaMSF Yanomami

Este notebook apresenta a exploração inicial dos dados utilizados no monitoramento de enchentes e riscos de malária na região Yanomami.

---

## Objetivos

- Carregar os dados históricos de malária, enchentes e alertas climáticos.
- Realizar uma análise exploratória dos principais indicadores.
- Visualizar padrões espaciais e temporais.

In [ ]:
# Importação das Bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Para visualização em mapas 
import folium

ModuleNotFoundError: No module named 'folium'

## Carregamento dos Dados

Nesta etapa, vamos carregar os arquivos CSV contendo os dados históricos de malária

In [ ]:
# Carregar dados de malária
malaria_df = pd.read_csv('/content/serie_historica_malaria_2013_2023.csv')


## Leitura e Visualização dos Dados de Vulnerabilidade a Inundações

Vamos carregar e visualizar o shapefile `Brasil_vulnerabilidade_inundacoes`, extra´dos do ANA que contém informações geográficas sobre áreas vulneráveis a enchentes no Brasil. Esses dados são fundamentais para analisar a sobreposição entre regiões de risco de inundação e casos de malária.

In [ ]:
# Leitura do shapefile de vulnerabilidade a inundações
import geopandas as gpd

# Caminho para o arquivo shapefile (.shp)
shapefile_path = 'content/Brasil_vulnerabilidade_inundacoes.shp'

# Carregar o shapefile como GeoDataFrame
gdf = gpd.read_file(shapefile_path)
gdf.head()

### Visualização Interativa das Áreas Vulneráveis

Podemos visualizar as áreas vulneráveis a inundações em um mapa interativo usando o método `.explore()` do GeoPandas.

In [ ]:
!pip install folium matplotlib mapclassify
# Visualização interativa das áreas vulneráveis
gdf.explore()

## Destaque das Áreas de Roraima e Amazonas

Vamos filtrar e visualizar apenas as regiões de vulnerabilidade a inundações localizadas nos estados de Roraima (RR) e Amazonas (AM), onde se encontra o território Yanomami.

In [ ]:
# Filtrar e visualizar áreas de Roraima e Amazonas

# Verifique o nome das colunas disponíveis
print(gdf.columns)

# Filtrar áreas de Roraima e Amazonas pela coluna 'UF'
gdf_rr_am = gdf[gdf['UF'].isin(['RR', 'AM'])]

# Remover 'geometry' da lista de colunas para tooltip/popup
cols = [col for col in gdf_rr_am.columns if col != 'geometry']

# Visualização interativa apenas dessas áreas
gdf_rr_am.explore(
    style_kwds={'fillColor': 'red', 'color': 'black', 'weight': 1, 'fillOpacity': 0.5},
    tooltip=cols,
    popup=True
)

## Consulta e Armazenamento de Alertas Climáticos (WeatherAPI)

As células abaixo permitem consultar alertas climáticos em tempo real (opcional), salvar os resultados em arquivo e carregar os dados salvos para garantir reprodutibilidade das análises.

In [ ]:
import os
import requests
import pandas as pd

API_KEY = os.getenv("WEATHER_API_KEY")  # Agora está correto!

lat, lon = -0.5, -64.5  # Aproximado para região Yanomami

url = f"http://api.weatherapi.com/v1/alerts.json?key={API_KEY}&q={lat},{lon}"
response = requests.get(url)
data = response.json()

# Extrair alertas
if 'alerts' in data and 'alert' in data['alerts']:
    alertas = data['alerts']['alert']
    if alertas:
        df_alertas = pd.DataFrame(alertas)
        display(df_alertas)
    else:
        print("Nenhum alerta ativo para esta região.")
else:
    print("Nenhum alerta retornado pela API.")

In [ ]:
# Salvar alertas em arquivo CSV/JSON
if 'df_alertas' in locals():
    df_alertas.to_csv('alertas_climaticos_atuais.csv', index=False)
    df_alertas.to_json('alertas_climaticos_atuais.json', orient='records', force_ascii=False)
    print("Alertas salvos em 'alertas_climaticos_atuais.csv' e 'alertas_climaticos_atuais.json'.")
else:
    print("Nenhum alerta para salvar. Execute a célula de consulta primeiro se desejar dados atualizados.")

In [ ]:
# Carregar alertas salvos de arquivo (reprodutível)
import pandas as pd

try:
    df_alertas = pd.read_csv('alertas_climaticos_atuais.csv')
    display(df_alertas)
except FileNotFoundError:
    print("Arquivo de alertas não encontrado. Execute a célula de consulta e salvamento primeiro, se necessário.")

## Dados sintéticos 
Devido à indisponibilidade de dados reais de enchentes e alertas climáticos no momento, optou-se por utilizar dados simulados (sintéticos) para viabilizar a análise exploratória e o treinamento dos modelos.

In [ ]:
import pandas as pd
import numpy as np

# Gerar 20 registros fictícios de enchentes
enchentes_df = pd.DataFrame({
    'municipio': np.random.choice(['Alto Alegre', 'Boa Vista', 'Mucajaí'], 20),
    'data': pd.date_range('2024-01-01', periods=20, freq='15D'),
    'area_alagada_km2': np.random.uniform(0.5, 10, 20),
    'imagem_path': [None]*20,
    'latitude': np.random.uniform(-3.5, -2.5, 20),
    'longitude': np.random.uniform(-63.5, -61.5, 20)
})

In [ ]:
alertas_df = pd.DataFrame({
    'tipo': np.random.choice(['Alerta de chuva', 'Alerta de vento', 'Alerta de calor'], 10),
    'descricao': np.random.choice(['Chuva intensa', 'Vento forte', 'Alta temperatura'], 10),
    'data_inicio': pd.date_range('2024-01-01', periods=10, freq='7D'),
    'data_fim': pd.date_range('2024-01-02', periods=10, freq='7D'),
    'municipio': np.random.choice(['Alto Alegre', 'Boa Vista', 'Mucajaí'], 10),
    'uf': ['RR']*10
})

## Visualização Inicial dos Dados

Vamos inspecionar as primeiras linhas de cada conjunto de dados para entender sua estrutura.

In [ ]:
# Visualizar as primeiras linhas dos dados
display(malaria_df.head())
display(enchentes_df.head())
display(alertas_df.head())

## Padronizar nomes de coluna para facilitar o restante do código


In [ ]:
if 'data_inicio' in alertas_df.columns:
    alertas_df = alertas_df.rename(columns={'data_inicio': 'data'})
if 'Ano' in malaria_df.columns:
    malaria_df = malaria_df.rename(columns={'Ano': 'data'})

# Agora, o restante do código pode usar .head() e manipular a coluna 'data' normalmente
display(malaria_df.head())
display(enchentes_df.head())
display(alertas_df.head())

## Análise Exploratória

Aqui, vamos analisar a distribuição dos casos de malária ao longo do tempo, a frequência de enchentes e os principais tipos de alertas climáticos.

In [ ]:
print(enchentes_df.columns)
print(alertas_df.columns)
print(malaria_df.columns)

In [ ]:
# Evolução dos casos de malária
malaria_df['data'] = pd.to_datetime(malaria_df['data'])
malaria_agg = malaria_df.groupby('data')['Área Indígena'].sum()
malaria_agg.plot(figsize=(12,6), title='Evolução dos casos de malária')
plt.xlabel('Data')
plt.ylabel('Área Indígena')
plt.show()

In [ ]:
# Frequência de enchentes por município
enchentes_df['data'] = pd.to_datetime(enchentes_df['data'])
floods_count = enchentes_df['municipio'].value_counts()
sns.barplot(x=floods_count.values, y=floods_count.index)
plt.title('Frequência de enchentes detectadas por município')
plt.xlabel('Número de enchentes')
plt.ylabel('Município')
plt.show()

In [ ]:
# Tipos de alertas climáticos mais comuns
alert_types = alertas_df['tipo'].value_counts()
sns.barplot(x=alert_types.values, y=alert_types.index)
plt.title('Tipos de alertas climáticos mais frequentes')
plt.xlabel('Quantidade')
plt.ylabel('Tipo de Alerta')
plt.show()

## Visualização Espacial 

Visualizar a localização das enchentes em um mapa interativo.

In [ ]:
# Mapa interativo das enchentes detectadas (ajuste lat/lon conforme seus dados)
m = folium.Map(location=[-3.5, -63.0], zoom_start=5)
for _, row in enchentes_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=6,
        popup=f"{row['municipio']} - {row['data']}",
        color="blue",
        fill=True,
        fill_color="blue"
    ).add_to(m)
m

## Conclusão

Devido ao pouco tempo ábil para busca e à indisponibilidade de dados reais de enchentes e alertas climáticos no momento, optou-se por utilizar dados simulados (sintéticos) para viabilizar a análise exploratória e o treinamento dos modelos. Essa abordagem permite validar o pipeline de dados e garantir que, assim que os dados reais estiverem disponíveis, o código estará pronto para ingestão, análise e visualização sem necessidade de grandes ajustes. Recomenda-se substituir os dados sintéticos pelos reais assim que possível, para obter resultados mais precisos e relevantes para a tomada de decisão. Esta análise inicial permite identificar tendências e padrões importantes para o desenvolvimento dos próximos módulos do projeto, como o modelo preditivo e as visualizações em tempo real na plataforma Streamlit.